In [1]:
using JuMP
using GLPK

Antes de mais nada, a seguinte convenção

$$\begin{array}{r|l}\mbox{Raw carrots} & c_1\\ \mbox{Baked potatoes} & c_2 \\ \mbox{Wheat bread} & c_3 \\ \mbox{Cheddar cheese} & c_4 \\ \mbox{Peanut butter} & c_5\end{array}$$

para a quantidade de cada ingrediente na dieta.

Queremos resolver o seguinte problema:
$$\min \sum_{i=1}^5 \mbox{Cost}_i(c_i)$$

sujeito a:

$$\sum_{i=1}^5 \mbox{Calories}(c_i) \geq 0\\
\sum_{i=1}^5 \mbox{Fat}(c_i) \geq 50\\
\sum_{i=1}^5 \mbox{Protein}(c_i) \geq 100\\
\sum_{i=1}^5 \mbox{Carbohydrates}(c_i) \geq 250\\
c_i \geq 0 
$$

In [2]:
cost(i) = [0.14, 0.12,0.2,0.75,0.15][i]
calories(i) = [23, 171, 65, 112, 188,0,cal][i]
fat(i) = [0.1, 0.2,0,9.3,16,50,fat][i]
protein(i) = [0.6, 3.7, 2.2, 7,7.7,100,prot][i]
carbo(i) = [6,30,13,0,2,250,carb][i]

carbo (generic function with 1 method)

In [3]:
model = Model(GLPK.Optimizer)
@variable(model, 0 <= c1)
@variable(model, 0 <= c2)
@variable(model, 0 <= c3)
@variable(model, 0 <= c4)
@variable(model, 0 <= c5)

@objective(model,Min,cost(1)*c1+cost(2)*c2+cost(3)*c3+cost(4)*c4+cost(5)*c5)

for func in [calories, fat, protein, carbo]
    @constraint(model,func(1)*c1+func(2)*c2+func(3)*c3+func(4)*c4+func(5)*c5 >= func(6))
end

UndefVarError: UndefVarError: cal not defined

In [4]:
optimize!(model)
objective_value(model)

0.0

In [5]:
[value(c1),value(c2),value(c3),value(c4),value(c5)]

5-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0

# Exercício 2

Antes de mais nada, a seguintes convenções:

$$\begin{array}{|r|l|}\hline\mbox{Produto} & \mbox{Quantidade produzida}\\ \hline\mbox{Skimmed milk} & p_1\\ \mbox{2%} & p_2 \\ \mbox{Whole milk} & p_3 \\ \mbox{Table cream} & p_4 \\ \mbox{Whipping cream} & p_5\\ \hline\end{array}$$

para a quantidade de cada produto produzido.

De início, vale notar que temos, efetivamente: $$500\mathcal{l} \times (1-0.037) + 250\mathcal{l} \times (1-0.049) = 719.25\mathcal{l}$$

de leite. Por conseguinte, temos:
$$500\mathcal{l} \times 0.037 + 250\mathcal{l} \times 0.049 = 30.75\mathcal{l}$$

de gordura totais.

Parece ser conveniente fazermos contas análogas da quantidade efetiva de leite e de gordura que cada produto irá precisar.

$$\begin{array}{|r|c|c|l|}\hline\mbox{Produto}& \mbox{Gordura (l)} & \mbox{Leite (l)} & \mbox{Profit (\$)}\\ \hline
p_1 & 0 & 2 & 0.1 \\ p_2 & 0.04 & 1.96 & 0.15  \\ p_3 & 0.08 & 1.92 & 0.2 \\ p_4 & 0.09 & 0.51 & 0.5 \\ p_5 & 0.135 & 0.165 & 1.2\\ \hline\end{array}$$ 

**Tornando nossa notação mais rigorosa.** 

Seja $\mathbf{p}\in \mathbb{R}^5$ a quantidade produzida de cada produto, $\mathbf{s}=\left[0.1\ 0.15\ 0.2\ 0.5\ 1.2\right]$ o lucro unitário para cada produto, $\mathbf{g}=\left[0\ 0.04\ 0.08\ 0.09\ 0.135\right]$ a gordura necessária para uma unidade de cada produto e $\mathbf{m}=\left[2\ 1.96\ 1.92\ 0.51\ 1.65\right]$ a quantidade de leite necessária para uma unidade de cada produto. 

Queremos resolver o seguinte problema:
$$\max_{\mathbf{p}} \mathbf{p}^\intercal\mathbf{s}$$

sujeito a:

$$\mathbf{p}^\intercal \mathbf{g} \leq 30.75\\ \mathbf{p}^\intercal \mathbf{m} \leq 719.25 \\ \mathbf{p} \geq 0$$

In [6]:
g = [0, 0.04, 0.08, 0.09, 0.135]
s = [0.1, 0.15, 0.2, 0.5, 1.2]
m = [2, 1.96, 1.92, 0.51, 1.65]

model_ex2 = Model(GLPK.Optimizer)

@variable(model_ex2, p[1:5] >= 0)
@constraint(model_ex2, sum(p.*g) <= 30.75)
@constraint(model_ex2, sum(p.*m) <= 719.25)

@objective(model_ex2, Max, sum(p.*s))

0.1 p[1] + 0.15 p[2] + 0.2 p[3] + 0.5 p[4] + 1.2 p[5]

In [7]:
optimize!(model_ex2)
objective_value(model_ex2)

290.50416666666666

In [8]:
value.(p)

5-element Array{Float64,1}:
 171.70833333333334
   0.0             
   0.0             
   0.0             
 227.77777777777777

### **b)** Considere agora que skimmed milk pode agora conter até $0.1%$ de gordura. Em que a formulação do problema muda?

Tudo muda que agora temos uma variável $t\in[0,\ 0.1]$ que gera a seguinte tabela:
$$\begin{array}{|r|c|c|l|}\hline\mbox{Produto}& \mbox{Gordura (l)} & \mbox{Leite (l)} & \mbox{Profit (\$)}\\ \hline
p_1 & 2t & 2(1-t) & 0.1 \\ p_2 & 0.04 & 1.96 & 0.15  \\ p_3 & 0.08 & 1.92 & 0.2 \\ p_4 & 0.09 & 0.51 & 0.5 \\ p_5 & 0.135 & 0.165 & 1.2\\ \hline\end{array}$$ 

O problema de otimização é o mesmo, acrescido de $t$ como variável de decisão.

Seja $\mathbf{p}\in \mathbb{R}^5$ a quantidade produzida de cada produto, $\mathbf{s}=\left[0.1\ 0.15\ 0.2\ 0.5\ 1.2\right]$ o lucro unitário para cada produto, $\mathbf{g}=\left[2t\ 0.04\ 0.08\ 0.09\ 0.135\right]$ a gordura necessária para uma unidade de cada produto e $\mathbf{m}=\left[2(1-t)\ 1.96\ 1.92\ 0.51\ 1.65\right]$ a quantidade de leite necessária para uma unidade de cada produto. 

**O problema agora é que deixou de ser otimização linear!!!**

Então precisamos de algum truque!

Adicionamos a variável $\lambda = 2p_1t$ do problema acima e apenas coloca a restrição

$$\max_{\{\mathbf{p},\lambda\}} \mathbf{p}^\intercal\mathbf{s}$$

sujeito a:

$$\mathbf{p}^\intercal \mathbf{g} \leq 30.75 - \lambda\\ \mathbf{p}^\intercal \mathbf{m} \leq  719.25 + \lambda\\ \mathbf{p} \geq 0 \\\ 0\leq \lambda \leq 0.002p_1 $$

In [19]:
g = [0, 0.04, 0.08, 0.09, 0.135]
s = [0.1, 0.15, 0.2, 0.5, 1.2]
m = [2, 1.96, 1.92, 0.51, 1.65]

model_ex2b = Model(GLPK.Optimizer)
@variable(model_ex2b, λ>=0)
@variable(model_ex2b, p[1:5] >= 0)
@constraint(model_ex2b, sum(p.*g) <= 30.75 - λ)
@constraint(model_ex2b, sum(p.*m) <= 719.25 + λ)

@objective(model_ex2b, Max, sum(p.*s))

optimize!(model_ex2b)
objective_value(model_ex2b)

290.50416666666666

In [20]:
value.(p) #tá certo???

5-element Array{Float64,1}:
 171.70833333333334
   0.0             
   0.0             
   0.0             
 227.77777777777777

In [21]:
value(λ) 

0.0

# Ideia: adicionar uma nova variável pra todo mundo

# Exercício 3

Antes de mais nada, a seguinte convenção:

$$\begin{array}{|c|c|}\hline\mbox{Nome} & \mbox{Variável} \\ \hline \mbox{Tom} & e_1\\
\mbox{Peter} & e_2 \\ \mbox{Nina} & e_3 \\ \mbox{Samir} & e_4 \\ \mbox{Gary} & e_5 \\ \mbox{Linda} & e_6 \\ \mbox{Bob} & e_7\\ \hline\end{array}
$$

O problema de programação linear é o seguinte:

$$\min_{\{e_1,e_2,e_3,e_4,e_5,e_6,e_7\}} \sum_{i=1}^7 e_i$$

Sujeito à:
$$\begin{align*}
e_1&\geq20000 \\
e_2,e_3,e_4&\geq 5000+e_1\\
e_5 &\geq e_1+e_2\\
e_6 &= e_5 + 200\\
e_3+e_4 &\geq 2(e_1+e_2)\\
e_7 &\geq e_2\\
e_7 &\geq e_4\\
e_7+e_2&\geq 60000\\
e_6 &\leq e_7+e_1\\
e_i&\geq 0
\end{align*}$$

In [11]:
model3 = Model(GLPK.Optimizer)

@variable(model3, e[1:7]>=0)
    
@objective(model3, Min, sum(e))

@constraint(model3, e[1] >= 20000)
@constraint(model3, e[2] >= 5000 + e[1])
@constraint(model3, e[3] >= 5000 + e[1])
@constraint(model3, e[4] >= 5000 + e[1])
@constraint(model3, e[5] >= e[1] + e[2])
@constraint(model3, e[3] + e[4] >= 2(e[1]+e[2]))
@constraint(model3, e[6] == (e[5] + 200))
@constraint(model3, e[7] >= e[2])
@constraint(model3, e[7] >= e[4])
@constraint(model3, e[7] + e[2] >= 60000)
@constraint(model3, e[6] <= e[7] + e[1])

e[6] - e[7] - e[1] ≤ 0.0

In [12]:
optimize!(model3)
objective_value(model3)

260200.0

In [13]:
value.(e)

7-element Array{Float64,1}:
 20000.0
 25000.0
 55000.0
 35000.0
 45000.0
 45200.0
 35000.0

Ou seja, teremos a seguinte tabela de rendimentos:
$$\begin{array}{|c|c|}\hline\mbox{Nome} & \mbox{Salário} \\ \hline \mbox{Tom} & 20000\\
\mbox{Peter} & 25000 \\ \mbox{Nina} & 55000 \\ \mbox{Samir} & 35000 \\ \mbox{Gary} & 45000 \\ \mbox{Linda} & 45200\\ \mbox{Bob} & 35000\\ \hline\end{array}
$$

# Agora minimize o salário máximo

In [14]:
model3 = Model(GLPK.Optimizer)

@variable(model3, e[1:7]>=0)
@variable(model3, aux) #tive spoilers
@objective(model3, Min, aux)
@constraint(model3, e[1] >= 20000)
@constraint(model3, e[2] >= 5000 + e[1])
@constraint(model3, e[3] >= 5000 + e[1])
@constraint(model3, e[4] >= 5000 + e[1])
@constraint(model3, e[5] >= e[1] + e[2])
@constraint(model3, e[3] + e[4] >= 2*(e[1]+e[2]))
@constraint(model3, e[6] == (e[5] + 200))
@constraint(model3, e[7] >= e[2])
@constraint(model3, e[7] >= e[4])
@constraint(model3, e[7] + e[2] >= 60000)
@constraint(model3, e[6] <= e[7] + e[1])

for i=1:7
    @constraint(model3, aux >= e[i])
end

In [15]:
optimize!(model3)
objective_value(model3)

45200.0

In [16]:
value.(e)

7-element Array{Float64,1}:
 20000.0
 25000.0
 45200.0
 44800.0
 45000.0
 45200.0
 44800.0

In [17]:
sum(value.(e))

270000.0